# Proof of Concept of different binary text classifiers on our data

In the world of ML, there is a vast amount of choice when it comes to which classification method to use.
In this notebook, I will be demonstrating how 4 of the most popular classifiers perform:

- <b>Multinomial Naive Bayes</b>, the 'punching bag' benchmark classifier of the ML world, using class membership probabilities found by feature vector weights, to predict the membership of a new data point,

- <b>Logistic Regression</b>, a method that uses the sigmoid function to transform a representation of how far a new data point lies from a decision boundary found via gradient descent to a class probability,

- <b>K-nearest-neighbours</b>, where the classification of X is a vote of the K nearest items to X,

- <b>SVM</b>, a method that tries to find a hyperplane to seperate classes by treating them as coordinates in an m dimensional space, m being the number of features.

The dataset that we are using in this notebook is very small, containing only 1600 data points, 800 of each class.

However, it is a good dataset to use to produce a POC in this notebook, because the feature extraction is fast, it is balanced, and the labels belong to the gold standard. This means we can cross examine multiple classifiers with multiple features in a fast manner to get a feel for how they perform.

In terms of producing a reliable model to serve on our API, it is not a good choice, as it does not generalize well.

Without further ado, let us begin.

## Data Processing & Feature Extraction

I start by importing some modules for data processing and manipulation, Numpy and Pandas.

I also import a helper function to process the raw data into a frame for us, for the sake of clarity.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from get_df import get_data_frame

Great. Let's take a look at our data:

In [ ]:
df = get_data_frame()
df.head()

Sweet. We have 3 columns: 
- Sentiment (0 is negative, 1 is positive)
- Review, our review text,
- Deceptive (0 is genuine, 1 is deceptive)

Sentiment and Deceptive are pre-labelled for us.
We will be focusing on the deceptive column, the label we wish to predict.

Let's seperate our data from the labels:

In [ ]:
X = df['review']
y = np.asarray(df['sentiment'], dtype=int)

These classifiers only work on numeric features, not the strings that our reviews are currently represented by. To represent our reviews as numeric features, we use a Bag of Words model. 

CountVectorizer takes our review and returns a $m$-dimensional array, where $m$ is our vocabularly size and $m_i$ is 1 if the word $i$ appears in the review, 0 if not.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

Lets take a look at the shape of our data after this transformation.

In [ ]:
print(cv.fit_transform(X).shape)

In this example, 1600 represents the number of reviews in our data, and 9571 the number of words (size of vocab.)
Let's see what happens if we remove stop words:

In [ ]:
cv = CountVectorizer(stop_words='english')
print(cv.fit_transform(X).shape)

That will help our classifier slightly. Stop words generally don't contribute anything to class membership, and add noise.

Here's what our first review looks like in count vector format:

In [ ]:
print((cv.fit_transform(X)[0]))

ie. the 4050'th word in the vocab appeared in the first review 3 times.

Now, let's apply another transformation. 

$Tf$ is term frequency, and corresponds to how many times a word appears in a review. The higher, more chances are that this particular review is relevant to this word.  

$df$ is the number of reviews the word occured in. The higher, the less we should weight the review because of the word. (If a word occurs in a large number of reviews , it wont play important role in finding out reviews relevant to  the word).

Therefore, we use the calculation $tf * \frac {1} {df} $.

This is known as $Tf-idf$, and provides a better representation of our data than just word counts. It weights words that are important to a review's classification higher.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()

Let's transform our first count vector and see what it looks like now.

In [ ]:
print(tfidf.fit_transform(cv.fit_transform(X)[0]))

## Finding Optimal Classifier Parameters

Now for the fun stuff! Let's import all of our classifiers listed above:

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm

Pipelines allow us to combine feature extractors with classifiers to make life easier.

In [ ]:
from sklearn.pipeline import Pipeline

### Naive Bayes

There aren't much different things we can try with Naive Bayes, however we can see the difference of unigrams to bigrams.

In [ ]:
cv_unigram = CountVectorizer(stop_words='english', ngram_range = (0, 1))
cv_bigram = CountVectorizer(stop_words='english', ngram_range = (0, 2))
mnb = MultinomialNB(alpha=2)

models = []

models.append(('Bigram MNB tfidf', Pipeline([ ('cv', cv_bigram), ('tfidf', tfidf), ('mnb', mnb) ])))
models.append(('Unigram MNB tfidf', Pipeline([ ('cv', cv_unigram), ('tfidf', tfidf), ('mnb', mnb) ])))


CV = 6
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model[0]
  accuracies = cross_val_score(model[1], X, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_df.groupby('model_name').accuracy.mean()

Looks like our Bigram-trained MNB is better.

### K-Nearest Neighbors

Let's see how many neighbors we should use for our k-NN classifier.

In [ ]:
from sklearn.model_selection import train_test_split

cv_knn = CountVectorizer(stop_words='english', ngram_range = (0, 1))

# Setup arrays to store training and test accuracies
neighbors = np.arange(5, 22)
train_accuracy =np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

# Split up the data into train and test for this problem
X_train = tfidf.fit_transform(cv_knn.fit_transform(X_train))
X_test = tfidf.transform(cv_knn.transform(X_test))

for i,k in enumerate(neighbors):
    #Setup a knn classifier with k neighbors
    knn = KNeighborsClassifier(n_neighbors=k)
    
    #Fit the model
    knn.fit(X_train, y_train)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(X_train, y_train)
    
    #Compute accuracy on the test set
    test_accuracy[i] = knn.score(X_test, y_test) 

In [ ]:
#Generate plot
plt.title('k-NN Varying number of neighbors')
plt.plot(neighbors, test_accuracy, label='Testing Accuracy')
plt.plot(neighbors, train_accuracy, label='Training accuracy')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.show()

18 seems to be our optimal number of neighbors.

### Support Vector Machine

There's a couple of different types of SVM with a bunch of different parameters. NuSVC is the classic SVC except you can tweak the number of support vectors. Let's compare the different types, using bigram tf-idf vectors as features.

In [ ]:
# Setup arrays to store models and param sets
cv = CountVectorizer(stop_words='english', ngram_range = (1, 2))

models = []

models.append(('SVC', Pipeline([ ('cv', cv), ('tfidf', tfidf), ('svm', svm.SVC(kernel="sigmoid", gamma='scale', random_state=42)) ])))
models.append(('LinearSVC', Pipeline([ ('cv', cv), ('tfidf', tfidf), ('svm', svm.LinearSVC(random_state=42)) ])))
models.append(('NuSVC', Pipeline([ ('cv', cv), ('tfidf', tfidf), ('svm', svm.NuSVC(kernel='sigmoid', gamma='scale', random_state=42)) ])))

CV = 6
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model[0]
  accuracies = cross_val_score(model[1], X, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

import seaborn as sns

sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, edgecolor="gray", linewidth=2)
plt.show()

cv_df.groupby('model_name').accuracy.mean()

Looks like the LinearSVC is our best option.

## Cross Classifier Comparison

Now we create all of our pipelines, one for each classifier.

In [ ]:
nbayes = Pipeline([ ('cv', cv), ('tfidf', tfidf), ('nbayes', MultinomialNB()) ])
logreg = Pipeline([ ('cv', cv), ('tfidf', tfidf), ('logreg', LogisticRegression(random_state=42, solver='liblinear')) ])
knn = Pipeline([ ('cv', cv_knn), ('tfidf', tfidf), ('knn', KNeighborsClassifier(n_neighbors=18)) ])
svc = Pipeline([ ('cv', cv), ('tfidf', tfidf), ('svm', svm.LinearSVC(random_state=42)) ])

models = {'Naive Bayes': nbayes, 'Log. Regression':logreg, 'k-NN':knn, 'SVC':svc}

For getting an accurate test/train split, we split our data up using 10 fold cross validation.

Now lets get our accuracy scores!

In [ ]:
CV = 6
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for name, model in models.items():
  model_name = name
  accuracies = cross_val_score(model, X, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])


Let's plot the 4 classifier accuracies against each other using the matplotlib library Seaborn.

In [ ]:
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

As we can see, the SVM worked best, whereas the k-NN algorithm does relatively poorly.